In [1]:
%load_ext autoreload
%autoreload 3

# import gconcord as cc
import gconcorde as cce
import numpy as np

np.random.seed(3)

# true precision matrix
omega = np.zeros((3,3), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 3)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
data = np.random.multivariate_normal([0]*3, sigma, 500)

# sample covariance matrix
S = np.cov(data, rowvar=False)

In [2]:
Omega_hat, hist = cce.pycceista(S, 0.3, epstol=1e-7, maxitr=500, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(Omega_hat)))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.22892494  0.74147241  0.        ]
 [ 0.74147241  0.93796959 -0.74109093]
 [ 0.         -0.74109093  1.23125868]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.00000000e+00 1.32622757e-07 2.22879809e+00]
 [3.00000000e+00 2.05221286e-07 2.22879809e+00]
 [5.00000000e+00 7.16891848e-08 2.22879809e+00]]

iterations: 110
successive norm diff: 7.168918e-08
objective: 2.228798e+00


In [3]:
omega_hat2, hist2 = cce.cceista(S, 0.3, epstol=1e-7, maxitr=500, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(omega_hat2.todense())))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist2[-3:])))
print("iterations: %d" % (len(hist2)))
print("successive norm diff: %e" % (hist2[-1,1]))
print("objective: %e" % (hist2[-1,2]))

omega_hat:
 [[ 1.22892494  0.74147241  0.        ]
 [ 0.74147241  0.93796959 -0.74109093]
 [ 0.         -0.74109093  1.23125868]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.00000000e+00 1.32622757e-07 2.22879809e+00]
 [3.00000000e+00 2.05221286e-07 2.22879809e+00]
 [5.00000000e+00 7.16891847e-08 2.22879809e+00]]

iterations: 110
successive norm diff: 7.168918e-08
objective: 2.228798e+00


In [4]:
Omega_hat, hist = cce.pycce(S, 0.3, epstol=1e-7, maxitr=500, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(Omega_hat)))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,0]))
print("objective: %e" % (hist[-1,1]))

omega_hat:
 [[ 1.22891056  0.74146191  0.        ]
 [ 0.74146191  0.93795358 -0.74107989]
 [ 0.         -0.74107989  1.2312441 ]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.16589988e-07 2.22879809e+00]
 [4.07570596e-07 2.22879809e+00]
 [3.98747634e-07 2.22879809e+00]]

iterations: 500
successive norm diff: 3.987476e-07
objective: 2.228798e+00


In [5]:
omega_hat2, hist2 = cce.cce(S, 0.3, epstol=1e-7, maxitr=500, penalize_diagonal=True)
print("omega_hat:\n %s\n" % (str(omega_hat2.todense())))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist2[-3:])))
print("iterations: %d" % (len(hist2)))
print("successive norm diff: %e" % (hist2[-1,0]))
print("objective: %e" % (hist2[-1,1]))

omega_hat:
 [[ 1.22891056  0.74146191  0.        ]
 [ 0.74146191  0.93795358 -0.74107989]
 [ 0.         -0.74107989  1.2312441 ]]

hist: (inner_iter_count, delta_subg, objective)
 [[4.16589988e-07 2.22879809e+00]
 [4.07570596e-07 2.22879809e+00]
 [3.98747634e-07 2.22879809e+00]]

iterations: 500
successive norm diff: 3.987476e-07
objective: 2.228798e+00
